In [ ]:
import os
from PIL import Image
import pandas as pd

In [ ]:
df = pd.read_csv("orange_cone.csv")

In [ ]:
df.describe().loc["count"]

##### 1. Current instances present in data are: 3842
##### 2. CSV data instances are: 25991
##### 3. CSV data instances required are: 16211

In [ ]:
df_o = df.iloc[-1264:].reset_index(drop=True)

In [ ]:
df_o.describe().loc["count"]

In [ ]:
df_o.index

### load image and crop image

In [ ]:
print("image path", df_o.image_path[0])
print("label path", df_o.path[0])

In [ ]:
def crop_and_save(image_path, label_path, box, img_output_folder, label_output_folder, class_label, counter):
    img = Image.open(image_path)
    cropped_img = img.crop(box)

    # Get the new bounding box coordinates in the cropped image
    new_box = (0, 0, box[2] - box[0], box[3] - box[1])
    
    # Create output folder if not exists
    os.makedirs(img_output_folder, exist_ok=True)
    os.makedirs(label_output_folder, exist_ok=True)
    
    # Save cropped image and labels to output folder
    img_output_folder = os.path.join(img_output_folder, os.path.basename(image_path))
    label_output_folder = os.path.join(label_output_folder, f"{os.path.basename(image_path)}.txt")

    # Image new name.
    image_new_name = img_new_name(img_output_folder, counter)
    
    cropped_img.save(image_new_name)

    # Write new bounding box coordinates to a text file
    text_file_path = label_output_folder
    label_name_update = label_new_name(text_file_path, counter)
    with open(text_file_path, 'w') as text_file:
        text_file.write(f"{class_label} {new_box[0]} {new_box[1]} {new_box[2]} {new_box[3]}")
    os.rename(text_file_path, label_name_update)

    print("Saved image: ", image_new_name)
    print("Saved label: ", label_name_update)

def img_new_name(path, counter):
    ext = os.path.basename(path).split(".")[-1]
    dir_name = os.path.dirname(path)
    parent_dir_name = os.path.basename(os.path.dirname(os.path.dirname(path)))
    new_image_path = os.path.join(dir_name, str(parent_dir_name) +"_"+ str(counter) +"."+ str(ext))
    return new_image_path

def label_new_name(path, counter):
    ext_1 = os.path.basename(path).split(".")[-1]
    ext_2 = os.path.basename(path).split(".")[-2]
    parent_dir_name = os.path.basename(os.path.dirname(os.path.dirname(path)))
    dir_name = os.path.dirname(path)
    new_label_path = os.path.join(dir_name, str(parent_dir_name) +"_"+ str(counter) +"_"+ str(ext_2) +"."+ str(ext_1))
    return new_label_path

In [ ]:
# Specify output folder for cropped images and labels
img_output_folder = 'orange_crop/images'
label_output_folder = 'orange_crop/labels'

# Iterate through rows and crop/save images
for index, row in df_o.iterrows():
    image_path = row['image_path']
    label_path = row['path']
    class_label = row["class_label"]
    box = (row['box_x1'], row['box_y1'], row['box_x2'], row['box_y2'])
    
    crop_and_save(image_path, label_path, box, img_output_folder, label_output_folder, class_label, index)
    
    print("Cropping and saving complete.", index)

#### Verify Bonding box on sample image

In [ ]:
from PIL import Image, ImageDraw
from IPython.display import display

# Load the original image using Pillow
original_image = Image.open('orange_crop/images/orange_crop_599.png')

# Define the new bounding box coordinates
x, y, width, height = 0, 0, 128, 160

# Create a drawing object
draw = ImageDraw.Draw(original_image)

# Draw a thicker rectangle on the image
draw.rectangle([x, y, x + width, y + height], outline="green", width=10)


# Display the image with the bounding box
display(original_image)
